## Orbit Properties

This script can be used to compute analytically a variety of orbit properties. The script was developed for the NuAncestor Mission Analysis. The code includes in particular the relations between the orbit period, semi-major axis, number of overflights above groundstations and the required tracking speed. 

Version 23/02/2025 by Blanca Crazzolara

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
MU_E = 	3.986004418e5 # km^3/s^2

def prograde_T(n, per = 23): # n = contacts per 23.56h
    if (per < 24):
        T = 3600*(23+56./60) / (1+n) # [s]
    else:
        T = 3600*(23+56./60) / (1-n) # [s]
    return T

def retrograde_T(n):
    T = 3600*(23+56./60) / (n-1) # [s]
    return T

def period(A): # period [s]
    T = 2*np.pi* np.sqrt(A**3/MU_E)  # [s]
    return T

def A(T):
    A = (MU_E * (T/2/np.pi)**2)**(1./3) # [km]
    return A

def alt(A):
    return A - 6371 # [km]

def period_h(A):
    return period(A)/3600.
def A_h(T):
    T = T*3600.
    return A(T)

def n(a, direction = "prograde"): # n = contacts/23.56h
    T = period(a) # [s]
    if(direction == "prograde" or direction == "pro" or direction == "p"):
        if (T/3600 < 24):
            N = (23+56./60)*3600  / T - 1
        else:
            N = 1 - (23+56./60)*3600  / T
    else:
        N = (23+56./60)*3600  / T + 1
    return N


In [ ]:
N_pro = np.linspace(0, 4, 30)
N_pro2 = np.linspace(0, 0.5, 20)
N_retro = np.linspace(2, 4, 50)


T_pro = prograde_T(N_pro)
T_pro2 = prograde_T(N_pro2, per = 25)
T_retro = retrograde_T(N_retro)

A_pro = A(T_pro)
A_pro2 = A(T_pro2)
A_retro = A(T_retro)

fig, ax = plt.subplots()
ax.plot(A_pro, N_pro/(23+56./60)*24, label = "prograde")
ax.plot(A_retro, N_retro/(23+56./60)*24, label = "retrograde")
ax.set_title("Equatorial Orbit")
plt.gca().set_yticks([0,1,2,3,4])
ax.set_xlabel("Semi-Major Axis [km]")
ax.set_ylabel("Overflights per day")
#ax.grid(True)
secax = ax.secondary_xaxis('top', functions=(period_h, A_h))
secax.set_xlabel('Period [h]')
plt.legend()
#plt.savefig("overflight_number_no_grid.png")



plt.figure()
plt.plot(T_pro/60/60, N_pro/(23+56./60)*24, label = "prograde")
plt.plot(T_retro/60/60, N_retro/(23+56./60)*24, label = "retrograde")
plt.title("Equatorial Orbit")
plt.gca().set_yticks([0,1,2,3,4])
plt.xlim(right = 24.9)
plt.xlabel("Period [h]")
plt.ylabel("Overflights per day")
plt.grid(True)
plt.legend()
#plt.savefig("overflight_number.png")

In [ ]:
MU = 398600.44189 # km^3 / s^2
R_E = 6371 # [km]
Omega = 2*np.pi / (3600*(23 + 56/60) ) # rad/s

def omega_obs(a, lat, inc = 0): # [rad/s]
    d = np.sqrt(a**2 + R_E**2 - 2 * R_E*a*np.cos(lat))
    if(inc == 0):
        w_obs = np.abs(a / d *(Omega - np.sqrt(MU/a**3)))
    elif(inc == 180):
        w_obs = np.abs(a / d *(Omega + np.sqrt(MU/a**3)))
    else:
        print("error due to non-defined formula")
    return w_obs*648000/np.pi

def omega_a_obs(a, lat, inc = 0): # [rad/s]
    d = np.sqrt(a**2 + R_E**2 - 2 * R_E*a*np.cos(lat))
    if(inc == 0):
        w_obs = 1./np.sin(lat) * np.abs(Omega - np.sqrt(MU/a**3))
    elif(inc == 180):
        w_obs = 1./np.sin(lat) * np.abs(Omega + np.sqrt(MU/a**3))
    else:
        print("error due to non-defined formula")
    return w_obs*648000/np.pi



In [ ]:

a_range = np.linspace(17000, 31000)
l_range = np.linspace(0, np.pi/2, 7)
L = len(l_range)
w_range = np.ones_like(a_range)
w_a_range = np.ones_like(a_range)

fig, ax = plt.subplots()
#plt.axhline(y=15, color='grey', linestyle='-')
#plt.axhline(y=30, color='grey', linestyle='--')

for i in range(len(l_range)):
    l = l_range[i]
    for j in range(len(a_range)):   
        w_range[j] = omega_obs(a_range[j], l, 0)
    ax.plot(a_range, w_range, label = rf"$\ell$ = {round(l/np.pi*180)}$\degree$", color=((L-i)/L, 0, i/L))

leg = ax.legend(ncol = 1)
leg.set_title("Latitude")

ax.set_xlabel("Semi-Major Axis [km]")
ax.set_ylabel("Tracking Speed [arcsec/s]")
ax.set_xlim(a_range[0], a_range[-1])
ax.set_ylim(0, top = 80)

secax = ax.secondary_xaxis('top', functions=(period_h, A_h))
secax.set_xlabel('Period [h]')
plt.savefig("tracking_speed_pro.png")

# Azimuthal velocity
fig, ax = plt.subplots()
#plt.axhline(y=15, color='grey', linestyle='-')
#plt.axhline(y=30, color='grey', linestyle='--')

for i in range(len(l_range)):
    l = l_range[i]
    if (l>0):
        for j in range(len(a_range)):   
            w_a_range[j] = omega_a_obs(a_range[j], l, 0)
        ax.plot(a_range, w_a_range,  label = rf"$\ell$ = {round(l/np.pi*180)}$\degree$", color=((L-i)/L, 0, i/L))

leg = ax.legend(ncol = 1)
leg.set_title("Latitude")
#leg.set_title("prograde   retrograde        ")

ax.set_xlabel("Semi-Major Axis [km]")
ax.set_ylabel("Azimuthal Tracking Speed [arcsec/s]")
ax.set_xlim(a_range[0], a_range[-1])
ax.set_ylim(0, top = 80)

secax = ax.secondary_xaxis('top', functions=(period_h, A_h))
secax.set_xlabel('Period [h]')
plt.savefig("tracking_speed_az_pro.png")


'''
for i in range(len(l_range)):
    l = l_range[i]
    for j in range(len(a_range)):   
        w_range[j] = omega_obs(a_range[j], l, i = 180)
    r = max(2*(L/2-i)/L,0)
    g = 1
    b = max(2*(0.5+i-L/2)/L,0)
    #ax.plot(a_range, w_range, label = rf"$\ell$ = {round(l/np.pi*180)}$\degree$", color=(r, g, b))
'''


In [ ]:
a_fit = 22914.976669523 
lat = 28.757 # -24.589 #28.757

print("afit: ", a_fit, " km")
print("max. track. speed: ", omega_obs(a_fit, 0), " arcsec/s")
print("track. speed: ", omega_obs(a_fit, lat*np.pi/180), " arcsec/s")
print("min. track. speed: ", omega_obs(a_fit, np.pi/2), " arcsec/s")
print("max. azimuthal speed: ", omega_a_obs(a_fit, lat*np.pi/180, inc = 0), "arcsec/s")
print("period: ", period_h(a_fit), " h")
print("number of overflights: ", n(a_fit, "p")/(23+56./60)*24)
print("Interval: ", (23+56./60)/n(a_fit, "p"))



In [ ]:
J2 = 1.08262668/1000
rho =  0.19910213e-6 # rad /s
c = 299792.458 # km / s


def v_max(a, e): # km/s
    v = np.sqrt(MU/a * (1+e) / (1-e))
    return v

def V_max(theta):
    a, e = theta
    return c - v_max(a, e)  #  > zero

def v_min(a, e): # km/s
    v = np.sqrt(MU/a * (1-e) / (1+e))
    return v

def d_min(a, e): # km
    return a*(1-e)

def D_min(theta):
    a, e = theta
    return d_min(a, e) - (R_E + 400) # > 0

def d_max(a, e): # km
    return a*(1+e)

def cos_i(a, e): # between -1 and 0
    cosi =  - 2*rho /(3*J2 * R_E**2 * np.sqrt(MU) ) * a**3.5 * (1-e**2)**2 
    return cosi

def inc(a, e):
    return np.arccos(cos_i(a,e))*180/np.pi

def Cos_i_0(theta):
    a, e = theta
    return -cos_i(a,e) # > 0

def Cos_i_1(theta):
    a, e = theta
    return cos_i(a,e) + 1  # > 0

def E_min(theta):
    a, e = theta
    return e # e >0

def E_max(theta):
    a,e  = theta
    return 1 - e # 1 > e

def ang(theta): # rad /s
    a,e = theta
    return v_min(a, e) / d_max(a,e)  # ang velocity at furthest point

def tot_ang(theta):
    dThet = ang(theta) # angular velocity of the satellit [rad/s]
    a, e = theta
    I = inc(a,e)/180*np.pi # rad

    return np.sqrt((np.cos(I)*dThet - Omega)**2 + (np.sin(I)*dThet)**2) # rad/s
    #return 1/(a*(1-e**2))**1.5 # ang vel at p 



In [ ]:
cons = ({'type': 'ineq', 'fun': V_max}, # c - vmax > 0
        {'type': 'ineq', 'fun': D_min}, # d_min - R_E  > 0
        {'type': 'ineq', 'fun': E_min}, # e > 0 
        {'type': 'ineq', 'fun': E_max}, # e < 1
        {'type': 'ineq', 'fun': Cos_i_0}, # cos i < 0
        {'type': 'ineq', 'fun': Cos_i_1}) # cos i > -1

theta_calc = minimize(tot_ang, (1.55*R_E, 0.2), constraints=cons)
a_opt, e_opt = theta_calc.x

print(f"Optimal semi-major axis: \t {round(a_opt,2)} km")
print(f"Optimal eccentricity:\t \t {round(e_opt,4)}")
print(f"Inclination: \t\t\t {round(inc(a_opt, e_opt))} deg")
print(f"Max vel: \t \t \t {round(v_max(a_opt, e_opt),2)} km/s")
print(f"Min Dist: \t \t\t {round(d_min(a_opt, e_opt),2)} km")
print(f"Min vel: \t \t \t {round(v_min(a_opt, e_opt),2)} km/s")
print(f"Max Dist: \t \t\t {round(d_max(a_opt, e_opt),2)} km")
print(f"Minimal value \t \t\t {round(tot_ang((a_opt, e_opt)),10)}")

In [ ]:
cons = ({'type': 'ineq', 'fun': V_max}, # c - vmax > 0
        {'type': 'ineq', 'fun': D_min}, # d_min - R_E  > 0
        {'type': 'ineq', 'fun': E_min}, # e > 0 
        {'type': 'ineq', 'fun': E_max}, # e < 1
        {'type': 'ineq', 'fun': Cos_i_0}, # cos i < 0
        {'type': 'ineq', 'fun': Cos_i_1}) # cos i > -1

theta_calc = minimize(tot_ang, (1.5*R_E, 0.1), constraints=cons)
a_opt, e_opt = theta_calc.x

print(f"Optimal semi-major axis: \t {round(a_opt,2)} km")
print(f"Optimal eccentricity:\t \t {round(e_opt,4)}")
print(f"Inclination: \t\t\t {round(inc(a_opt, e_opt))} deg")
print(f"Max vel: \t \t \t {round(v_max(a_opt, e_opt),2)} km/s")
print(f"Min Dist: \t \t\t {round(d_min(a_opt, e_opt),2)} km")
print(f"Min vel: \t \t \t {round(v_min(a_opt, e_opt),2)} km/s")
print(f"Max Dist: \t \t\t {round(d_max(a_opt, e_opt),2)} km")
print(f"Minimal value \t \t\t {round(tot_ang((a_opt, e_opt)),10)}")

In [ ]:
cons = ({'type': 'ineq', 'fun': V_max}, # c - vmax > 0
        {'type': 'ineq', 'fun': D_min}, # d_min - R_E  > 0
        {'type': 'ineq', 'fun': E_min}, # e > 0 
        {'type': 'ineq', 'fun': E_max}, # e < 1
        {'type': 'ineq', 'fun': Cos_i_0}, # cos i < 0
        {'type': 'ineq', 'fun': Cos_i_1}) # cos i > -1

theta_calc = minimize(tot_ang, (1.5*R_E, 0.1), constraints=cons)
a_opt, e_opt = theta_calc.x
print(cos_i(a_opt, e_opt))

print(f"Optimal semi-major axis: \t {round(a_opt,2)} km")
print(f"Optimal eccentricity:\t \t {round(e_opt,4)}")
print(f"Inclination: \t\t\t {round(inc(a_opt, e_opt))} deg")
print(f"Max vel: \t \t \t {round(v_max(a_opt, e_opt),2)} km/s")
print(f"Min Dist: \t \t\t {round(d_min(a_opt, e_opt),2)} km")
print(f"Min vel: \t \t \t {round(v_min(a_opt, e_opt),2)} km/s")
print(f"Max Dist: \t \t\t {round(d_max(a_opt, e_opt),2)} km")
print(f"Minimal value \t \t\t {round(tot_ang((a_opt, e_opt)),10)}")

In [ ]:
a_range = np.linspace(R_E, 3*R_E, 100)
e_range = np.linspace(0, 1, 100)
A, E = np.meshgrid(a_range, e_range)
Ang = 0.*np.ones_like(A)

for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        a = A[i,j]
        e = E[i,j]
        cos_I = cos_i(a,e)
        min_dist = d_min(a,e)
        v_maximum = v_max(a,e)
        if (cos_I < 0 and cos_I >= -1 and inc(a,e) > 180):
            #print("not valid due to high inc")
            Ang[i,j] = 6000
        elif(cos_I > 0 and cos_I < 1):
            #print("not valid due to low inc")
            Ang[i,j] = 8000
        elif(cos_I < -1 or cos_I > 1):
            #print("unphysical inclination")
            Ang[i,j] = 10000
        elif(min_dist < R_E+400):
            #print("not valid due to impact")
            Ang[i,j] = 12000
        elif(v_maximum > c ): # no SSO
            #print("not valid combiation due to high vel")
            Ang[i,j] = 14000
        else:
            Ang[i,j] = tot_ang((a,e))/np.pi*180*3600*24 # deg/24h
        
            

In [ ]:
plt.pcolor(A, E, Ang)
plt.colorbar()
plt.xlabel("semi-major axis")
plt.ylabel("eccentricity")
indx = np.unravel_index(Ang.argmin(), Ang.shape)
print(indx)
print(round(A[indx],2), round(E[indx],4), round(inc(A[indx], E[indx]),2))
print(np.min(Ang))

In [ ]:
plt.pcolor(A, E, Ang)
plt.colorbar()
plt.xlabel("semi-major axis")
plt.ylabel("eccentricity")
indx = np.unravel_index(Ang.argmin(), Ang.shape)
print(indx)
print(round(A[indx],2), round(E[indx],4), round(inc(A[indx], E[indx]),2))
print(np.min(Ang))

In [ ]:
Ang[0,0]